In [ ]:
import socket
import numpy as np
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import joblib

# Function to perform a basic port scan on a target host
def port_scan(target_ip, port_range):
    open_ports = []
    for port in port_range:
        sock = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
        sock.settimeout(1)
        result = sock.connect_ex((target_ip, port))
        if result == 0:
            open_ports.append(port)
        sock.close()
    return open_ports

# Feature extraction function for ML model
def extract_port_features(open_ports):
    features = []
    for port in open_ports:
        features.append([port, port % 256])  # Just an example, could be more sophisticated
    return np.array(features)

# Function to train the machine learning model based on port scan data
def train_port_scan_model(dataset):
    X = dataset.drop('label', axis=1)  # Features (open ports, etc.)
    y = dataset['label']  # 1 for vulnerable, 0 for secure

    # Standardizing features
    scaler = StandardScaler()
    X_scaled = scaler.fit_transform(X)

    # Splitting dataset into training and test sets
    X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)

    # Random Forest model for classification
    clf = RandomForestClassifier(n_estimators=100, random_state=42)
    clf.fit(X_train, y_train)

    # Save model and scaler
    joblib.dump(clf, 'port_scan_model.pkl')
    joblib.dump(scaler, 'port_scan_scaler.pkl')

    return clf, scaler

# Function to make insights and predictions from port scan data
def analyze_ports(target_ip, clf, scaler, port_range):
    open_ports = port_scan(target_ip, port_range)
    features = extract_port_features(open_ports)
    features_scaled = scaler.transform(features)

    predictions = clf.predict(features_scaled)
    return predictions

# Function to perform port scan with insights
def port_scan_with_insights(target_ip, port_range):
    # Load the trained model and scaler
    clf = joblib.load('port_scan_model.pkl')
    scaler = joblib.load('port_scan_scaler.pkl')

    predictions = analyze_ports(target_ip, clf, scaler, port_range)

    print(f"Scan results for {target_ip}:")
    for port, prediction in zip(port_range, predictions):
        if prediction == 1:
            print(f"Port {port} is vulnerable")
        else:
            print(f"Port {port} is secure")

# Example usage
target_ip = "192.168.1.10"
port_range = range(1, 65536)  # Scanning all ports
port_scan_with_insights(target_ip, port_range)